# A Tale of Two Investor Types:
## How the Interaction of Forward-looking and Backward-looking Investors Explains the Non-Random Walk of Asset Markets
### Forward-looking = **Long Term Investors**
### Backward-looking = **Extrapolators**

In [ ]:
%load_ext autoreload
%autoreload 2

import polars as pl
import altair as alt
from bubbles.agents import *
from bubbles.core import *
from bubbles.dz import dz
from itables import (init_notebook_mode, show)

init_notebook_mode(all_interactive=True)
alt.theme.enable('latimes')

## Model Parameters

In [ ]:
mp = Market(seed=0) #1337194922

extrapolator = Extrapolator.new()
long_term_investor = LongTermInvestor.new()
investors = [extrapolator, long_term_investor]

print(mp)
print(extrapolator)
print(long_term_investor)

In [ ]:
df = data_table(mp, investors)
df = df.select([
    pl.col('^.*$').cast(pl.Float64).round(6)
])
show(df[12 * mp.history_length + 1:], maxBytes=0)
    

### Percent of ending wealth held by long term inverstors

In [ ]:
wealth_y = df.get_column("Wealth y")[-1]
wealth_x = df.get_column("Wealth x")[-1]
wealth_y / (wealth_y +  wealth_x)

In [ ]:
chart_data = df.select([
    'Month',
    pl.col('Price Idx').alias('Price Index'),
    pl.col('Fair Value')
]).unpivot(
    index=['Month'],
    on=['Price Index', 'Fair Value'],
    variable_name='Metric',
    value_name='Value'
)

alt.Chart(chart_data).mark_line().encode(
    x='Month',
    y='Value',
    color=alt.Color('Metric:N', scale=alt.Scale(
        domain=['Price Index', 'Fair Value'],
    ))
).properties(
    width=700,
    height=500,
    title="Stock Market Price Index vs. Fair Value"
)

In [ ]:
(df.plot.line(x='Month', y='Premium')
   .encode(
       color=alt.datum('Premium')  # This creates a legend entry for Premium
   )
   .properties(width=700, height=500, title="Premium / Discount")
)